In [1]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
pip install lightgbm

In [ ]:
pip install catboost

In [14]:
import pandas as pd 
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report

import xgboost as xgb

import catboost as cat



In [3]:
df_order_jan = pd.read_csv("orders_before_jan_labeled.csv")
df_order_jan['label'].value_counts()

0    882569
4      8883
3      6846
2      4522
1      3061
Name: label, dtype: int64

In [4]:
# unbalanced data remove extra 0s 
df_10000_0 = df_order_jan.loc[df_order_jan['label']==0].sample(n=10000, axis=0, random_state=1)
df_balanced_jan = df_10000_0.append(df_order_jan.loc[df_order_jan['label']!=0])
df_balanced_jan['label'].value_counts()

0    10000
4     8883
3     6846
2     4522
1     3061
Name: label, dtype: int64

In [5]:
# load other dataset
df_cat_hierarchy = pd.read_csv('category_hierarchy.csv', delimiter='|',error_bad_lines=False)
df_items = pd.read_csv('items.csv',sep='|',error_bad_lines=False)
# fill NA values in category column -- 4300 is the category with null values
df_items["categories"] = df_items["categories"].fillna('[4300]')
df_items

,itemID,brand,feature_1,feature_2,feature_3,feature_4,feature_5,categories
0,22665,861,4,0,490,2,66,"[2890, 855, 3908, 3909]"
1,28640,1366,10,1,537,0,101,[4300]
2,13526,1090,10,0,511,0,0,"[3270, 163, 284, 1694, 12, 3837, 2422, 3595, 3..."
3,21399,1090,10,1,511,0,0,[3270]
4,8504,768,4,1,484,0,66,[2470]
...,...,...,...,...,...,...,...,...
32771,12036,1383,10,0,503,0,101,"[3502, 1088, 2004, 691, 3422, 2308, 308, 3688,..."
32772,22709,567,4,2,491,3,66,"[3900, 3905, 3903, 3910, 1807, 274, 1467, 914]"
32773,32322,1385,10,1,500,0,37,"[2686, 2609, 725]"
32774,19118,1113,6,2,491,0,117,"[3900, 3910, 3903, 2473, 914, 274]"


In [6]:
# transform categories from str to int list
df_items['categories'] = df_items['categories'].map(lambda x: list(map(int,x.split("[")[1].split("]")[0].split(","))))

In [7]:
# from categories list generate parent_categories
df_indexed = df_cat_hierarchy.set_index(['category'])
df_indexed

,parent_category
category,
0,75
1,1499
2,1082
3,3498
4,1623
...,...
4295,3898
4296,3898
4297,3898


In [8]:
df_indexed.loc[[2890, 855, 3908, 3909]]['parent_category']

category
2890    2832
855     1178
3908    3898
3909    3898
Name: parent_category, dtype: int64

In [9]:
# set parent of null as null
df_indexed.loc[4300] = [4300]
df_indexed

,parent_category
category,
0,75
1,1499
2,1082
3,3498
4,1623
...,...
4296,3898
4297,3898
4298,3898


In [10]:
# parent_categories generate
df_items['parent_categories'] = df_items['categories'].map(lambda x : df_indexed.loc[x]['parent_category'].tolist())

In [11]:
df_items

,itemID,brand,feature_1,feature_2,feature_3,feature_4,feature_5,categories,parent_categories
0,22665,861,4,0,490,2,66,"[2890, 855, 3908, 3909]","[2832, 1178, 3898, 3898]"
1,28640,1366,10,1,537,0,101,[4300],[4300]
2,13526,1090,10,0,511,0,0,"[3270, 163, 284, 1694, 12, 3837, 2422, 3595, 3...","[1420, 3860, 600, 600, 3241, 3241, 3241, 600, ..."
3,21399,1090,10,1,511,0,0,[3270],[1420]
4,8504,768,4,1,484,0,66,[2470],[2566]
...,...,...,...,...,...,...,...,...,...
32771,12036,1383,10,0,503,0,101,"[3502, 1088, 2004, 691, 3422, 2308, 308, 3688,...","[3860, 458, 458, 458, 458, 322, 600, 600, 600,..."
32772,22709,567,4,2,491,3,66,"[3900, 3905, 3903, 3910, 1807, 274, 1467, 914]","[3898, 3898, 3898, 3898, 1072, 2920, 2475, 3565]"
32773,32322,1385,10,1,500,0,37,"[2686, 2609, 725]","[2917, 1175, 725, 1735]"
32774,19118,1113,6,2,491,0,117,"[3900, 3910, 3903, 2473, 914, 274]","[3898, 3898, 3898, 2566, 3565, 2920]"


In [25]:
# Joining df_orders_jan dataframe and df_items dataframe
merge_jan = pd.merge(df_items, df_balanced_jan, how='inner', on='itemID')
#change the date coulumn's datayte to datetime datatype
merge_jan['date'] = pd.to_datetime(merge_jan['date'])
merge_jan = merge_jan.set_index('date')

In [26]:
y = merge_jan['label']
X = merge_jan.drop(columns='label')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)
X_train

,itemID,brand,feature_1,feature_2,feature_3,feature_4,feature_5,categories,parent_categories,userID,order
date,,,,,,,,,,,
2020-11-23,17749,147,4,0,491,3,66,"[218, 2767, 2558, 3915]","[2006, 3550, 2894, 3898]",31095,1
2020-07-11,11250,1383,10,0,502,0,178,"[3639, 1088, 691]","[3860, 458, 458]",44941,1
2020-10-03,4421,1496,4,0,114,3,16,"[945, 914, 3871]","[918, 3565, 1092]",3634,4
2020-12-14,31953,1040,10,0,236,0,84,"[3459, 3107, 3587]","[3056, 2703, 3829]",39144,1
2020-11-23,11118,110,4,1,282,0,16,[218],[2006],26235,2
...,...,...,...,...,...,...,...,...,...,...,...
2020-07-16,22606,615,10,0,385,0,147,[614],[2457],23136,1
2020-08-10,3612,1065,4,0,536,3,144,"[2396, 324, 581, 1390, 2910, 2474, 3915, 3914,...","[3027, 3617, 3617, 2762, 2036, 1511, 3898, 389...",1642,1
2020-12-28,21625,1111,4,0,487,0,144,"[2256, 3042]","[370, 370]",29100,2


In [22]:
X_train['date'].dtypes

dtype('<M8[ns]')

In [15]:
# upload submission dataset
sub_jan = pd.read_csv("submission_jan.csv", sep="|")
sub_jan

,userID,itemID,prediction
0,0,20664,NaN
1,0,28231,NaN
2,13,2690,NaN
3,15,1299,NaN
4,15,20968,NaN
...,...,...,...
9935,46118,20106,NaN
9936,46124,19677,NaN
9937,46125,12878,NaN
9938,46127,7963,NaN


In [27]:
# baseline(didn't consider time series)--xgboost
def xgb_train(X_train, y_train, X_test, y_test, verbose=True):
    model_xgb = xgb.XGBClassifier(
        max_depth=10, # raw8
        n_estimators=1000,
        min_child_weight=300, 
        colsample_bytree=0.8, 
        subsample=0.8, 
        eta=0.3,    
        seed=42        
    )

    model_xgb.fit(
        X_train, 
        y_train,
        eval_metric='auc',
        eval_set=[(X_train, y_train), (X_test, y_test)],
        verbose=verbose,
        early_stopping_rounds=10 # 早停法，如果auc在10epoch没有进步就stop
    )
    print(model_xgb.best_score)
    return model_xgb

In [28]:
model_xgb = xgb_train(X_train, y_train, X_test, y_test, verbose=False)

ValueError: DataFrame.dtypes for data must be int, float, bool or category.  When
categorical type is supplied, DMatrix parameter `enable_categorical` must
be set to `True`. Invalid columns:categories, parent_categories

In [ ]:
prob = model_xgb.predict_proba(test_data)

submission['prediction'] = pd.Series(prob[:,1])
# submission.drop(['origin'], axis=1, inplace=True)
submission.to_csv('submission_xgb.csv', index=False)

In [ ]:
# baseline1: fill 0
# .fillna(0, inplace=True)